In [1]:
import requests
import pandas as pd
import random
from bs4 import BeautifulSoup

In [2]:
jobTitle = "Junior Python"
location = "Madrid"
workType = ["en remoto", "hibrido"]

In [22]:
jobTitleFormatted = jobTitle.replace(' ', "%2B")
numberResults = "25"
jobs_list_link = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={jobTitleFormatted}&location={location}&start={numberResults}"

response_jlg = requests.get(jobs_list_link)

jobs_list_data = response_jlg.text
jobs_list_soup = BeautifulSoup(jobs_list_data, "html.parser")
jobs_page = jobs_list_soup.find_all("li")

In [23]:
job_id_list = []
for job in jobs_page:
    base_card_div = job.find("div", {"class": "base-card"})
    job_id = base_card_div.get("data-entity-urn").split(":")[3]
    job_id_list.append(job_id)
print(job_id_list)

['4371426144', '4370681847', '4367265991', '4365313666', '4372322252', '4371577061', '4363952174', '4370692601', '4370253002', '4372328220']


In [27]:
jobs_list = []
for job_id in job_id_list:
    job_especifics_link = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    response_je = requests.get(job_especifics_link)
    response_je.raise_for_status() 
    job_soup = BeautifulSoup(response_je.text , "html.parser")
    job_info = {}

    # Job Title
    try:
        tag = job_soup.select_one("a.topcard__link h2.top-card-layout__title")
        job_info["job_title"] = tag.text.strip()
    except:
        raise ValueError("Job tag not found in job page")
    
    
    # Company Info
    if tag := job_soup.find("a", class_="topcard__org-name-link"):
        job_info["company_name"] = tag.text.strip()
        job_info["company_link"] = tag.get("href")    
    else:
        raise ValueError("Company tag not found in job page")

    # We asume all esential data is present so we add it to the array
    jobs_list.append(job_info)
    # Number of applicants
    try:
        tag = job_soup.select_one("figcaption.num-applicants__caption")
        job_info["num_applicants"] = tag.text.strip()
    except:
        job_info["num_applicants"] = None
    
    # Time posted
    try:
        tag = job_soup.select_one("span.posted-time-ago__text")
        job_info["time_posted"] = tag.text.strip()
    except:
        job_info["time_posted"] = None



In [28]:
jobs_df = pd.DataFrame(jobs_list)
jobs_df

,job_title,company_name,company_link,num_applicants,time_posted
0,Junior Project Manager,VML THE COCKTAIL,https://es.linkedin.com/company/vml-thecocktai...,Over 200 applicants,4 days ago
1,Delegado Dermatología - Madrid,IQVIA,https://www.linkedin.com/company/iqvia?trk=pub...,None,5 days ago
2,Técnico/a de Formación - Madrid,rpc España,https://es.linkedin.com/company/rpc-espana?trk...,None,2 weeks ago
3,Jornadas Jóvenes Talentos,LLYC,https://es.linkedin.com/company/llyc?trk=publi...,Over 200 applicants,3 weeks ago
4,Graduate software engineer,Evernote,https://www.linkedin.com/company/evernote?trk=...,None,23 hours ago
5,¿Eres recién titulado? Únete a Accenture Techn...,Accenture España,https://es.linkedin.com/company/accenture-espa...,None,3 days ago
6,Talento Joven Financiero Junior,Movistar Prosegur Alarmas,https://es.linkedin.com/company/movistar-prose...,Over 200 applicants,4 days ago
7,Accounts payable Position,Howden,https://uk.linkedin.com/company/howden-insuran...,None,5 days ago
8,Administración de Personal y Archivo TRAINEE ·...,SEPHORA,https://fr.linkedin.com/company/sephora?trk=pu...,Over 200 applicants,1 week ago
9,Graduate software engineer,Meetup,https://www.linkedin.com/company/meetup?trk=pu...,None,23 hours ago
